In [1]:
from rich import print
import supriya

In [4]:
server = supriya.Server().boot()

ServerCannotBoot: ERROR: Input sample rate is 16000, but output is 44100. Mismatched sample rates are not supported. To disable input, set the number of input channels to 0.

In [ ]:
with server.at():
    with server.add_synthdefs(supriya.default):
        synth = server.add_synth(supriya.default)

In [7]:
print(server.status)

StatusInfo(
    actual_sample_rate=44101.305459719784,
    average_cpu_usage=0.12219543009996414,
    group_count=2,
    peak_cpu_usage=0.3182348906993866,
    synth_count=1,
    synthdef_count=33,
    target_sample_rate=44100.0,
    ugen_count=20
)

In [8]:
print(server.query_tree())

QueryTreeGroup(
    node_id=0,
    annotation=None,
    children=[
        QueryTreeGroup(
            node_id=1,
            annotation=None,
            children=[
                QueryTreeSynth(
                    node_id=1000,
                    annotation=None,
                    synthdef_name='supriya:default',
                    controls=[
                        QueryTreeControl(name_or_index='out', value=0.0),
                        QueryTreeControl(name_or_index='amplitude', value=0.10000000149011612),
                        QueryTreeControl(name_or_index='frequency', value=440.0),
                        QueryTreeControl(name_or_index='gate', value=1.0),
                        QueryTreeControl(name_or_index='pan', value=0.5)
                    ]
                )
            ]
        )
    ]
)

In [9]:
print(server.root_node)

RootNode(
    context=<Server ONLINE [/Applications/SuperCollider.app/Contents/Resources/scsynth -R 0 -l 1 -u 57110]>,
    id_=0,
    parallel=False
)

In [10]:
print(server.default_group)

Group(
    context=<Server ONLINE [/Applications/SuperCollider.app/Contents/Resources/scsynth -R 0 -l 1 -u 57110]>,
    id_=1,
    parallel=False
)

In [6]:
import time
import supriya
from supriya.ugens import PinkNoise, Line, Out
# import sounddevice as sd # Uncomment if you installed sounddevice

def boot_supriya_bluetooth():
    # --- 1. PRE-CONFIGURATION LOGIC ---
    print("--> [1/4] CONFIGURING AUDIO DRIVERS...")

    headphones = "Bose AE2 SoundLink"
    laptop_mic = "MacBook Pro Microphone"
    
    # Default to None (System Default) unless we find the Bose
    selected_output = None 
    
    # Optional: Dynamic Detection Logic
    # If you haven't installed 'sounddevice', you can just hardcode selected_output = headphones
    # if you know it's connected.
    # devices = [d['name'] for d in sd.query_devices()]
    # if headphones in devices:
    #     selected_output = headphones
    #     print(f"    >> DETECTED: {headphones}")
    # else:
    #     print("    >> HEADPHONES NOT FOUND. USING DEFAULT.")

    # For now, let's assume you want to force it like your SC script:
    selected_output = headphones

    # --- 2. INSTANTIATE OPTIONS (IMMUTABLE) ---
    # We pass all arguments here. We cannot change them later.
    options = supriya.Options(
        sample_rate=44100,
        input_device=laptop_mic,   # Critical: Force Input to Laptop
        output_device=selected_output,
        
        # Audio Bus configuration
        input_bus_channel_count=2,
        output_bus_channel_count=2,
        
        # Memory / Buffer settings (Mirroring your SC settings)
        buffer_count=262144,       # 1024 * 256
        memory_size=262144,        # 8192 * 32
        maximum_node_count=32768,  # 1024 * 32
        wire_buffer_count=512
    )

    # --- 3. BOOT ---
    print("--> [2/4] BOOTING SERVER...")
    server = supriya.Server()
    server.boot(options=options)

    # --- 4. THE "AUDIO PRIMER" ---
    # This keeps the bluetooth clock alive
    print("--> [3/4] WAKING UP BLUETOOTH HARDWARE...")
    
    with supriya.SynthDefBuilder() as builder:
        # Pink Noise at -80db (approx 0.0001 amplitude)
        source = PinkNoise.ar() * 0.0001 
        # 1.5 second envelope
        env = Line.kr(start=1, stop=0, duration=1.5, done_action=2)
        out = source * env
        Out.ar(bus=0, source=[out, out])
        
    primer_def = builder.build()
    _ = server.add_synth(primer_def)
    
    # Allow audio to flow to stabilize the clock
    time.sleep(2.0)

    # --- 5. READY ---
    print("--> [4/4] SUPRIYA READY.")
    print(f"    Input: {server.options.input_device}")
    print(f"    Output: {server.options.output_device}")
    
    return server

s = boot_supriya_bluetooth()

--> [1/4] CONFIGURING AUDIO DRIVERS...

--> [2/4] BOOTING SERVER...

ServerCannotBoot: 

In [7]:
import supriya

def debug_boot():
    print("--> ATTEMPTING DEBUG BOOT...")
    
    # 1. Minimal Options (No memory flags yet, just audio routing)
    options = supriya.Options(
        sample_rate=44100,
        input_device="MacBook Pro Microphone", 
        output_device="Bose AE2 SoundLink",
        input_bus_channel_count=2,
        output_bus_channel_count=2
    )

    server = supriya.Server()

    try:
        # 2. Try to boot
        server.boot(options=options)
        print("--> SUCCESS! Server is running.")
        return server
        
    except Exception as e:
        print("\n!!! BOOT FAILED !!!")
        # 3. This is the magic line that reveals WHY it failed:
        if hasattr(e, "process_protocol"):
             print(e.process_protocol.error_text)
        else:
            print(e)
            
        return None

s = debug_boot()

--> ATTEMPTING DEBUG BOOT...

!!! BOOT FAILED !!!

In [ ]:
supriya.scsynth

pathlib.Path

In [ ]:
import supriya
import time
from supriya.ugens import PinkNoise, Line, Out

def boot_with_fixes():
    # --- UPDATE THESE STRINGS FROM THE DUMP SCRIPT ABOVE ---
    # Common Intel Mac Internal Mic name:
    my_input = "MacBook Pro Microphone" 
    # my_input = "MacBook Pro Microphone" # (Apple Silicon)
    
    my_output = "Bose AE2 SoundLink"
    
    # Try 48k first. Bluetooth prefers it.
    my_rate = 48000 
    
    print(f"--> CONFIGURING: In='{my_input}', Out='{my_output}', Rate={my_rate}")

    options = supriya.Options(
        sample_rate=my_rate, 
        input_device=my_input,
        output_device=my_output,
        input_bus_channel_count=1,
        output_bus_channel_count=2,
        # Reduced buffer count to ensure memory isn't the crash cause
        buffer_count=16384 
    )

    server = supriya.Server()
    
    try:
        server.boot(options=options)
        print("--> SUCCESS! Server booted.")
    except Exception as e:
        print("\n--> FAILED.")
        if hasattr(e, "process_protocol"):
             print(f"Server Error Log:\n{e.process_protocol.error_text}")
        else:
            print(f"Python Error: {e}")
        return None

    # --- WAKE UP BLUETOOTH ---
    # Only run this if boot succeeded
    # print("--> PRIMING AUDIO...")
    # with supriya.SynthDefBuilder() as builder:
    #     source = PinkNoise.ar() * 0.0001
    #     env = Line.kr(start=1, stop=0, duration=1.0, done_action=2)
    #     Out.ar(bus=0, source=[source * env] * 2)
    
    # _ = server.add_synth(builder.build())
    # time.sleep(1.5)
    
    return server

# Run
s = boot_with_fixes()

--> CONFIGURING: In='MacBook Pro Microphone', Out='Bose AE2 SoundLink', Rate=48000

--> SUCCESS! Server booted.

--> PRIMING AUDIO...

/Users/ramseybarghouti/Documents/strudel-music/.venv/lib/python3.12/site-packages/supriya/contexts/realtime.py:236: FailWarning: /s_new SynthDef not found
  warnings.warn(


In [20]:
s.quit()

<Server OFFLINE [/Applications/SuperCollider.app/Contents/Resources/scsynth -H 'MacBook Pro Microphone' 'Bose AE2 SoundLink' -R 0 -S 48000 -b 16384 -i 1 -l 1 -o 2 -u 57110]>

In [ ]:
with s.at():
    with s.add_synthdefs(supriya.default):
        synth = s.add_synth(supriya.default)

In [28]:

# Common Intel Mac Internal Mic name:
my_input = "MacBook Pro Microphone" 
# my_input = "MacBook Pro Microphone" # (Apple Silicon)

my_output = "Bose AE2 SoundLink"

# Try 48k first. Bluetooth prefers it.
my_rate = 48000 

options = supriya.Options(
    sample_rate=my_rate, 
    input_device=my_input,
    output_device=my_output,
    input_bus_channel_count=1,
    output_bus_channel_count=2,
    # Reduced buffer count to ensure memory isn't the crash cause
    buffer_count=16384 
)

server = supriya.Server()

server.boot(options=options)

with server.at():
    with server.add_synthdefs(supriya.default):
        synth = server.add_synth(supriya.default)

time.sleep(2.0)
print(server.status)
print(server.query_tree())
print(server.root_node)
print(server.default_group)
print(server.options)
server.quit()
print(server.status)
print(server.status)
print(server.status)

StatusInfo(
    actual_sample_rate=44100.00004433154,
    average_cpu_usage=0.1292067915201187,
    group_count=2,
    peak_cpu_usage=0.12750239670276642,
    synth_count=1,
    synthdef_count=33,
    target_sample_rate=44100.0,
    ugen_count=20
)

QueryTreeGroup(
    node_id=0,
    annotation=None,
    children=[
        QueryTreeGroup(
            node_id=1,
            annotation=None,
            children=[
                QueryTreeSynth(
                    node_id=1000,
                    annotation=None,
                    synthdef_name='supriya:default',
                    controls=[
                        QueryTreeControl(name_or_index='out', value=0.0),
                        QueryTreeControl(name_or_index='amplitude', value=0.10000000149011612),
                        QueryTreeControl(name_or_index='frequency', value=440.0),
                        QueryTreeControl(name_or_index='gate', value=1.0),
                        QueryTreeControl(name_or_index='pan', value=0.5)
                    ]
                )
            ]
        )
    ]
)

RootNode(
    context=<Server ONLINE [/Applications/SuperCollider.app/Contents/Resources/scsynth -H 'MacBook Pro Microphone' 
'Bose AE2 SoundLink' -R 0 -S 48000 -b 16384 -i 1 -l 1 -o 2 -u 57110]>,
    id_=0,
    parallel=False
)

Group(
    context=<Server ONLINE [/Applications/SuperCollider.app/Contents/Resources/scsynth -H 'MacBook Pro Microphone' 
'Bose AE2 SoundLink' -R 0 -S 48000 -b 16384 -i 1 -l 1 -o 2 -u 57110]>,
    id_=1,
    parallel=False
)

Options(
    audio_bus_channel_count=1024,
    block_size=64,
    buffer_count=16384,
    control_bus_channel_count=16384,
    executable=None,
    hardware_buffer_size=None,
    initial_node_id=1000,
    input_bus_channel_count=1,
    input_device='MacBook Pro Microphone',
    input_stream_mask='',
    ip_address='127.0.0.1',
    load_synthdefs=True,
    maximum_logins=1,
    maximum_node_count=1024,
    maximum_synthdef_count=1024,
    memory_locking=False,
    memory_size=8192,
    output_bus_channel_count=2,
    output_device='Bose AE2 SoundLink',
    output_stream_mask='',
    password=None,
    port=57110,
    protocol='udp',
    random_number_generator_count=64,
    realtime=True,
    restricted_path=None,
    safety_clip=None,
    sample_rate=48000,
    threads=6,
    ugen_plugins_path=None,
    verbosity=0,
    wire_buffer_count=64,
    zero_configuration=False
)

StatusInfo(
    actual_sample_rate=44100.00004433154,
    average_cpu_usage=0.1292067915201187,
    group_count=2,
    peak_cpu_usage=0.12750239670276642,
    synth_count=1,
    synthdef_count=33,
    target_sample_rate=44100.0,
    ugen_count=20
)

StatusInfo(
    actual_sample_rate=44100.00004433154,
    average_cpu_usage=0.1292067915201187,
    group_count=2,
    peak_cpu_usage=0.12750239670276642,
    synth_count=1,
    synthdef_count=33,
    target_sample_rate=44100.0,
    ugen_count=20
)

StatusInfo(
    actual_sample_rate=44100.00004433154,
    average_cpu_usage=0.1292067915201187,
    group_count=2,
    peak_cpu_usage=0.12750239670276642,
    synth_count=1,
    synthdef_count=33,
    target_sample_rate=44100.0,
    ugen_count=20
)

In [31]:
supriya.scsynth.find()

PosixPath('/Applications/SuperCollider.app/Contents/Resources/scsynth')